In [5]:
# Import the module to mount Google Drive in Google Colab
from google.colab import drive

# Mount Google Drive to the Colab environment
# This will prompt you to authorize access by clicking a link and pasting an authorization code
drive.mount('/content/drive')

# After mounting, your Drive files will be accessible at:
# /content/drive/MyDrive/


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Install OpenCV (used for image/video processing and object detection)
!pip install opencv-python


In [7]:
# Download the pre-trained SSD MobileNet V3 model (COCO-trained) from TensorFlow's model zoo
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v3_large_coco_2020_01_14.tar.gz


--2025-05-13 11:07:23--  http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v3_large_coco_2020_01_14.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.253.117.207, 142.250.99.207, 142.250.107.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|172.253.117.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49318766 (47M) [application/gzip]
Saving to: ‘ssd_mobilenet_v3_large_coco_2020_01_14.tar.gz.1’

ssd_mobilenet_v3_la 100%[===================>]  47.03M   153MB/s    in 0.3s    

2025-05-13 11:07:24 (153 MB/s) - ‘ssd_mobilenet_v3_large_coco_2020_01_14.tar.gz.1’ saved [49318766/49318766]



In [8]:
# Download the model config file required to use the SSD MobileNet V3 model with OpenCV
!wget https://huggingface.co/spaces/BilalSardar/Object-Color-Detection-in-Video/resolve/main/ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt


--2025-05-13 11:07:31--  https://huggingface.co/spaces/BilalSardar/Object-Color-Detection-in-Video/resolve/main/ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt
Resolving huggingface.co (huggingface.co)... 3.163.189.74, 3.163.189.90, 3.163.189.37, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182178 (178K) [text/plain]
Saving to: ‘ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt’

ssd_mobilenet_v3_la 100%[===================>] 177.91K  --.-KB/s    in 0.06s   

2025-05-13 11:07:31 (2.73 MB/s) - ‘ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt’ saved [182178/182178]



In [9]:
# Extract the contents of the downloaded .tar.gz model file
!tar -xvf ssd_mobilenet_v3_large_coco_2020_01_14.tar.gz


ssd_mobilenet_v3_large_coco_2020_01_14/pipeline.config
ssd_mobilenet_v3_large_coco_2020_01_14/model.ckpt.index
ssd_mobilenet_v3_large_coco_2020_01_14/model.ckpt.data-00000-of-00001
ssd_mobilenet_v3_large_coco_2020_01_14/model.tflite
ssd_mobilenet_v3_large_coco_2020_01_14/frozen_inference_graph.pb
ssd_mobilenet_v3_large_coco_2020_01_14/checkpoint
ssd_mobilenet_v3_large_coco_2020_01_14/model.ckpt.meta
ssd_mobilenet_v3_large_coco_2020_01_14/


In [10]:
# Download the list of class names used in the COCO dataset (used for labeling detected objects)
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names


--2025-05-13 11:07:42--  https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 625 [text/plain]
Saving to: ‘coco.names’

coco.names          100%[===================>]     625  --.-KB/s    in 0s      

2025-05-13 11:07:42 (48.2 MB/s) - ‘coco.names’ saved [625/625]



In [11]:
# List all files in the current working directory
!ls -l

# List all files inside the extracted model folder to confirm contents (e.g., .pb file)
!ls -l ssd_mobilenet_v3_large_coco_2020_01_14/


total 96524
-rw-r--r-- 1 root   root       625 May 13 11:07 coco.names
drwx------ 6 root   root      4096 May 13 11:06 drive
drwxr-xr-x 1 root   root      4096 May  9 13:41 sample_data
drwxr-xr-t 2 382342 89939     4096 Jan 15  2020 ssd_mobilenet_v3_large_coco_2020_01_14
-rw-r--r-- 1 root   root    182178 May 13 11:07 ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt
-rw-r--r-- 1 root   root  49318766 Jan 15  2020 ssd_mobilenet_v3_large_coco_2020_01_14.tar.gz
-rw-r--r-- 1 root   root  49318766 Jan 15  2020 ssd_mobilenet_v3_large_coco_2020_01_14.tar.gz.1
total 58840
-rw-r--r-- 1 382342 89939       77 Jan 15  2020 checkpoint
-rw-r--r-- 1 382342 89939 13448454 Jan 15  2020 frozen_inference_graph.pb
-rw-r--r-- 1 382342 89939 26117824 Jan 15  2020 model.ckpt.data-00000-of-00001
-rw-r--r-- 1 382342 89939    11691 Jan 15  2020 model.ckpt.index
-rw-r--r-- 1 382342 89939  7623069 Jan 15  2020 model.ckpt.meta
-rw-r--r-- 1 382342 89939 13027584 Jan 15  2020 model.tflite
-rw-r--r-- 1 382342 89939     4

In [12]:
import cv2
import numpy as np

# Load class labels from the COCO dataset
classNames = []
with open('coco.names', 'r') as f:
    classNames = f.read().rstrip('\n').split('\n')

# Load the pre-trained SSD MobileNet V3 model using OpenCV's DNN module
net = cv2.dnn_DetectionModel(
    'ssd_mobilenet_v3_large_coco_2020_01_14/frozen_inference_graph.pb',  # model weights
    'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'  # model config
)

# Configure input settings for the model
net.setInputSize(320, 320)                      # Resize input image to 320x320
net.setInputScale(1.0 / 127.5)                  # Normalize pixel values
net.setInputMean((127.5, 127.5, 127.5))         # Subtract mean for normalization
net.setInputSwapRB(True)                        # Convert BGR to RGB


< cv2.dnn.Model 0x7ddf795a3df0>

In [13]:
# Print the total number of class labels (should be 80 for COCO dataset)
print(len(classNames))

# Print the first 5 class labels to verify correct loading
print(classNames[:5])

80
['person', 'bicycle', 'car', 'motorbike', 'aeroplane']


In [14]:
# Replace with your image path from Google Drive
image_path = '/content/drive/MyDrive/boy with car.jpg'

# Read the image using OpenCV
img = cv2.imread(image_path)

# Run object detection on the image
classIds, confs, bbox = net.detect(img, confThreshold=0.5)

# Draw bounding boxes and labels on the detected objects
for classId, confidence, box in zip(classIds.flatten(), confs.flatten(), bbox):
    cv2.rectangle(img, box, color=(0, 255, 0), thickness=2)  # Draw rectangle
    cv2.putText(img, classNames[classId - 1].upper(), (box[0]+10, box[1]+30),  # Add label
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

# Display the result using matplotlib (converts BGR to RGB for correct color display)
from matplotlib import pyplot as plt
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()


error: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [15]:
import cv2
import numpy as np
import os

# Path to the input video from Google Drive
video_path = '/content/drive/MyDrive/Fydp/Tejgaon_20230518_104610_2.mp4'

# Path to save the output video
output_path = '/content/drive/MyDrive/detected_output.mp4'

# Open the video file
cap = cv2.VideoCapture(video_path)

# Check if the video opened successfully
if not cap.isOpened():
    print(" Error: Could not open input video file.")
else:
    print(" Input video loaded successfully.")

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Resolution: {frame_width}x{frame_height}")
print(f"FPS: {fps}")
print(f"Total frames: {frame_count}")

# Define video writer with matching resolution and frame rate
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Better compatibility with .mp4
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Process the video frame-by-frame
while True:
    ret, frame = cap.read()
    if not ret:
        break  # End of video

    # Run object detection
    detections = net.detect(frame, confThreshold=0.5)

    # Unpack detections safely
    if len(detections) == 3:
        classIds, confs, bbox = detections
    else:
        classIds, confs, bbox = [], [], []

    # Draw detections
    for classId, confidence, box in zip(np.array(classIds).flatten(),
                                        np.array(confs).flatten(), bbox):
        label_index = classId - 1
        if 0 <= label_index < len(classNames):
            label = classNames[label_index].upper()
        else:
            label = f'CLASS_{classId}'

        cv2.rectangle(frame, box, color=(0, 255, 0), thickness=2)
        cv2.putText(frame, label, (box[0] + 10, box[1] + 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Write processed frame to output video
    out.write(frame)

# Release all resources
cap.release()
out.release()
print(f" Processing complete. Output saved to: {output_path}")


 Error: Could not open input video file.
Resolution: 0x0
FPS: 0.0
Total frames: 0
 Processing complete. Output saved to: /content/drive/MyDrive/detected_output.mp4
